In [1]:
import pandas as pd
import numpy as np
import random
import os

In [44]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed=27
seed_everything(seed)

In [45]:
df_train = pd.read_csv('/content/drive/MyDrive/데이콘/웹로그기반조회수예측/open-3/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/데이콘/웹로그기반조회수예측/open-3/test.csv')
df_train = df_train.drop(['sessionID','userID'],axis=1)
df_test = df_test.drop(['sessionID','userID'],axis=1)

In [46]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252289 entries, 0 to 252288
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   TARGET               252289 non-null  float64
 1   browser              252289 non-null  object 
 2   OS                   252289 non-null  object 
 3   device               252289 non-null  object 
 4   new                  252289 non-null  int64  
 5   quality              252289 non-null  float64
 6   duration             252289 non-null  float64
 7   bounced              252289 non-null  int64  
 8   transaction          252289 non-null  float64
 9   transaction_revenue  252289 non-null  float64
 10  continent            252289 non-null  object 
 11  subcontinent         252289 non-null  object 
 12  country              252289 non-null  object 
 13  traffic_source       252289 non-null  object 
 14  traffic_medium       252289 non-null  object 
 15  keyword          

In [47]:
df_train.corr()

<ipython-input-47-81897f4db25c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_train.corr()


,TARGET,new,quality,duration,bounced,transaction,transaction_revenue
TARGET,1.000000,-0.066609,0.632723,0.595340,-0.464659,0.322097,0.180687
new,-0.066609,1.000000,-0.182483,-0.087505,0.053208,-0.072169,-0.042463
quality,0.632723,-0.182483,1.000000,0.459374,-0.249452,0.475446,0.246404
duration,0.595340,-0.087505,0.459374,1.000000,-0.337534,0.211381,0.123186
bounced,-0.464659,0.053208,-0.249452,-0.337534,1.000000,-0.088368,-0.042052
transaction,0.322097,-0.072169,0.475446,0.211381,-0.088368,1.000000,0.462039
transaction_revenue,0.180687,-0.042463,0.246404,0.123186,-0.042052,0.462039,1.000000


In [48]:
df_train.fillna('NAN', inplace=True)
df_test.fillna('NAN', inplace=True)


In [49]:
x_train = df_train.drop('TARGET', axis=1)
y_train = df_train['TARGET']

In [50]:
from sklearn.preprocessing import OrdinalEncoder

features_to_encode = x_train.select_dtypes(include=['object']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ordinal_encoder.fit(x_train[features_to_encode])
x_train[features_to_encode] = ordinal_encoder.transform(x_train[features_to_encode])
df_test[features_to_encode] = ordinal_encoder.transform(df_test[features_to_encode])

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [74]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

params = {'num_leaves': 64,
          'learning_rate': 0.001,
          'n_estimators': 3000,
          'random_state': 42}


model = LGBMRegressor(**params)

model.fit(X_train,
          y_train,
          eval_set=[(X_valid, y_valid)],
          callbacks=[lgb.log_evaluation(100)])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009749 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1359
[LightGBM] [Info] Number of data points in the train set: 201831, number of used features: 16
[LightGBM] [Info] Start training from score 3.593254
[100]	valid_0's l2: 25.0791
[200]	valid_0's l2: 21.8612
[300]	valid_0's l2: 19.2156
[400]	valid_0's l2: 17.0387
[500]	valid_0's l2: 15.2493
[600]	valid_0's l2: 13.7769
[700]	valid_0's l2: 12.5581
[800]	valid_0's l2: 11.5408
[900]	valid_0's l2: 10.701
[1000]	valid_0's l2: 10.0001
[1100]	valid_0's l2: 9.41496
[1200]	valid_0's l2: 8.92784
[1300]	valid_0's l2: 8.52319
[1400]	valid_0's l2: 8.1896
[1500]	valid_0's l2: 7.91249
[1600]	valid_0's l2: 7.681
[1700]	valid_0's l2: 7.48484
[1800]	valid_0's l2: 7.31782
[1900]	valid_0's l2: 7.17733
[2000]	valid_0's l2: 7.06131
[2100]	valid_0's l2: 6.961

LGBMRegressor(learning_rate=0.001, n_estimators=3000, num_leaves=64,
              random_state=42)

In [69]:
pred = model.predict(df_test)
pred = [0 if i < 0 else i for i in pred]

In [70]:
pred =[0 if i<0 else i for i in pred]
pred

[26.854326574478566,
 1.1333288542916184,
 2.798946211927175,
 4.206807908113266,
 10.872183776153623,
 4.03957959624958,
 10.39378266321928,
 3.5492065324246616,
 1.1333288542916184,
 1.1333288542916184,
 3.074086886527635,
 1.1333288542916184,
 4.155898055171939,
 14.104722938870896,
 1.1333288542916184,
 1.1333288542916184,
 1.1333288542916184,
 1.1333288542916184,
 1.1333288542916184,
 1.1333288542916184,
 5.406644369070428,
 2.764350344437595,
 13.255933430496022,
 1.1333288542916184,
 2.241231642719821,
 7.2385665375373485,
 1.1333288542916184,
 12.680898219641564,
 1.1333288542916184,
 1.1333288542916184,
 3.346826996069132,
 25.369002250374017,
 1.1333288542916184,
 4.080566132133451,
 6.67783860463919,
 2.245515481968522,
 2.251273333717942,
 3.390726788669724,
 8.853040852192084,
 6.901562170990941,
 1.1333288542916184,
 3.0661231171585155,
 1.1333288542916184,
 5.038643733582961,
 4.37184396222884,
 1.1333288542916184,
 2.528894530559751,
 2.1531392281465083,
 1.133328854291

In [71]:
df_submit = pd.read_csv('/content/drive/MyDrive/데이콘/웹로그기반조회수예측/open-3/sample_submission.csv')
df_submit['TARGET'] = pred

In [72]:
df_submit.head()

,sessionID,TARGET
0,SESSION_252289,26.854327
1,SESSION_252290,1.133329
2,SESSION_252291,2.798946
3,SESSION_252292,4.206808
4,SESSION_252293,10.872184


In [73]:
df_submit.to_csv("sample_submission.csv", index=False)